In [ ]:
# Cell 1: Import Libraries
import osmnx as ox
import networkx as nx
import random
from shapely.geometry import Point
import math
import numpy as np # Added for potential future numerical operations

In [ ]:
# Cell 2: Define Hospital Class
class Hospital:
    def __init__(self, location, capacity, latlong):
        self.location = location  # Node ID in the graph
        self.capacity = capacity
        self.latlong = latlong # Latitude and longitude coordinates
        self.beginning = capacity # Store initial capacity for resetting

In [ ]:
# Cell 3: Define Victim Class
class Victim:
    def __init__(self, location, rating, latlong):
        self.location = location  # Node ID in the graph
        self.rating = rating      # 'R' (Red/Critical), 'G' (Green), 'U' (Unknown)
        self.visited = False      # Flag to track if visited by a vehicle
        self.latlong = latlong    # Latitude and longitude coordinates

In [ ]:
# Cell 4: Define Vehicle Class
class Vehicle:
    def __init__(self, vehicle_type, max_risk, location, capacity, index, latlong):
        self.type = vehicle_type     # e.g., 'ambulance', 'helicopter', 'rescue_truck'
        self.max_risk = max_risk     # Maximum risk level the vehicle can tolerate on a path
        self.location = location     # Current node ID in the graph
        self.capacity = capacity     # Max number of victims vehicle can carry
        self.current_amt = 0         # Current number of victims in the vehicle
        self.route = []              # Planned sequence of nodes for the vehicle's route
        self.risk_scores = []        # Stores risk scores for segments of the route
        self.index = index           # Unique identifier for the vehicle
        self.latlong = latlong       # Latitude and longitude coordinates
        self.assigned_victims = []   # List of Victim objects assigned to this vehicle

In [ ]:
# Cell 5: Helper Function to Get Nearest Node
def get_nearest_node(graph, latlong):
    """Finds the closest node in the OSMnx graph for given latitude and longitude."""
    point = Point(latlong[1], latlong[0]) # Shapely Point expects (longitude, latitude)
    return ox.nearest_nodes(graph, point.x, point.y)

In [ ]:
# Cell 6: Vehicle Risk Calculation Function
def vehicle_risk(u, v, data, vehicle_type, vehicle_max_risk=5, alpha=0.5, beta=0.5):
    """Calculates a combined score for traversing an edge based on time and risk.
       Returns infinity if the edge's danger exceeds vehicle_max_risk.
    """
    # Default edge speed for general network if 'maxspeed' is not present
    # OSMnx automatically imputes missing speeds for 'drive' network_type if speed column not available.
    # We'll rely on the 'travel_time' attribute which OSMnx adds if it's not present.
    travel_time = data.get('travel_time', 0) # Assumes osmnx has calculated this

    # Get danger score from edge data, default to 0 if not present
    temp_danger = data.get('danger', 0)

    # Check if the edge is traversable based on vehicle's max_risk
    if temp_danger > vehicle_max_risk:
        return float('inf'), 0, 0  # Not traversable for this vehicle type

    # Calculate combined weight (score) for the edge
    # The formula can be adjusted based on real-world requirements
    combined_weight = alpha * travel_time + beta * temp_danger

    return combined_weight, travel_time, temp_danger

In [ ]:
# Cell 7: Graph Initialization and Data Generation
# Define the bounding box for the operational area
# Using (west, south, east, north) for bbox parameter as per OSMnx v2 migration guide
bbox_coords = (-95.6, 29.5, -95.2, 29.8) # Roughly Sugar Land, TX area

# Get a graph of the road network within the bounding box
print("Downloading graph...")
# G = ox.graph_from_bbox(north, south, east, west, network_type='drive') # Deprecated
G = ox.graph_from_bbox(bbox_coords[3], bbox_coords[1], bbox_coords[2], bbox_coords[0], network_type='drive') # Corrected for v2.0.0
print("Graph downloaded.")

# Impute missing travel times (speeds) if not already present
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

# Add random danger scores to edges
for u, v, k, data in G.edges(keys=True, data=True):
    data['danger'] = random.randint(0, 10) # Danger score from 0 to 10

all_nodes = list(G.nodes())

# Generate Hospitals
num_hospitals = 3
hospitals = []
print(f"Generating {num_hospitals} hospitals...")
for i in range(num_hospitals):
    # Get random coordinates within the bounding box
    rand_lat = random.uniform(bbox_coords[1], bbox_coords[3])
    rand_lon = random.uniform(bbox_coords[0], bbox_coords[2])
    latlong = (rand_lat, rand_lon)
    node = get_nearest_node(G, latlong)
    capacity = random.randint(5, 15) # Random capacity between 5 and 15
    hospitals.append(Hospital(node, capacity, latlong))
    print(f"  Hospital {i+1}: Node {node}, Capacity {capacity}, LatLong {latlong}")

# Generate Victims
num_victims = 15
victims = []
victim_ratings = ['R'] * 3 + ['G'] * 7 + ['U'] * 5 # 3 Red, 7 Green, 5 Unknown
random.shuffle(victim_ratings)

print(f"Generating {num_victims} victims...")
for i in range(num_victims):
    rand_lat = random.uniform(bbox_coords[1], bbox_coords[3])
    rand_lon = random.uniform(bbox_coords[0], bbox_coords[2])
    latlong = (rand_lat, rand_lon)
    node = get_nearest_node(G, latlong)
    rating = victim_ratings[i]
    victims.append(Victim(node, rating, latlong))
    print(f"  Victim {i+1}: Node {node}, Rating {rating}, LatLong {latlong}")

# Generate Vehicles
num_vehicles = 4
vehicles = []
vehicle_types = ['ambulance', 'helicopter', 'rescue_truck'] # Example types
type_props = {'ambulance': {'max_risk': 3, 'capacity': 2, 'speed_factor': 1.0}, # Speed factor applied to travel time
              'helicopter': {'max_risk': 10, 'capacity': 1, 'speed_factor': 0.5}, # Helicopters ignore roads, faster
              'rescue_truck': {'max_risk': 7, 'capacity': 3, 'speed_factor': 0.8}}

print(f"Generating {num_vehicles} vehicles...")
for i in range(num_vehicles):
    v_type = random.choice(vehicle_types)
    props = type_props[v_type]
    # Start all vehicles at a random hospital location
    start_hospital = random.choice(hospitals)
    vehicles.append(Vehicle(v_type, props['max_risk'], start_hospital.location, 
                            props['capacity'], i, start_hospital.latlong))
    print(f"  Vehicle {i+1}: Type {v_type}, Max Risk {props['max_risk']}, Capacity {props['capacity']}, Start Node {start_hospital.location}")

print("\n--- Initialization Complete ---\n")

In [ ]:
# Cell 8: Pre-calculate Vehicle Matrices (Shortest Paths)
vehicle_matrices = {}

nodes_of_interest = set([v.location for v in victims] + [h.location for h in hospitals] + [veh.location for veh in vehicles])
nodes_of_interest = list(nodes_of_interest)

print("Calculating shortest path matrices for each vehicle type...")
for vehicle_type in type_props.keys():
    vehicle_info = type_props[vehicle_type]
    max_risk_for_type = vehicle_info['max_risk']
    speed_factor_for_type = vehicle_info['speed_factor']

    type_matrix = {'score': {}, 'time': {}, 'risk': {}, 'path': {}}

    for source_node in nodes_of_interest:
        type_matrix['score'][source_node] = {}
        type_matrix['time'][source_node] = {}
        type_matrix['risk'][source_node] = {}
        type_matrix['path'][source_node] = {}

        for target_node in nodes_of_interest:
            if source_node == target_node:
                type_matrix['score'][source_node][target_node] = 0
                type_matrix['time'][source_node][target_node] = 0
                type_matrix['risk'][source_node][target_node] = 0
                type_matrix['path'][source_node][target_node] = [source_node]
                continue

            # Use Dijkstra's algorithm to find the shortest path
            # The 'weight' parameter here refers to the combined score from vehicle_risk
            try:
                path = nx.shortest_path(G, source=source_node, target=target_node, 
                                         weight=lambda u, v, data: vehicle_risk(u, v, data, vehicle_type, max_risk_for_type)[0])
                
                total_time = 0
                total_risk = 0
                for i in range(len(path) - 1):
                    u, v = path[i], path[i+1]
                    # Find the specific edge (handle multi-edges)
                    edge_data = G.get_edge_data(u, v)
                    # Take the first edge if multiple exist, or iterate if specific logic needed
                    edge_key = next(iter(edge_data)) # Get the key of the first edge
                    data = edge_data[edge_key]
                    
                    # Recalculate based on the edge data
                    _, time, risk = vehicle_risk(u, v, data, vehicle_type, max_risk_for_type)
                    total_time += time
                    total_risk += risk
                
                # Apply speed factor for vehicle type
                total_time *= speed_factor_for_type
                
                # Calculate the final score for the path
                # Adjust weights as needed (e.g., risk is more penalizing)
                path_score = (total_time / 100) + (0.2 * total_risk)

                type_matrix['score'][source_node][target_node] = path_score
                type_matrix['time'][source_node][target_node] = total_time
                type_matrix['risk'][source_node][target_node] = total_risk
                type_matrix['path'][source_node][target_node] = path
            except nx.NetworkXNoPath:
                # No path exists or path involves edges exceeding max_risk
                type_matrix['score'][source_node][target_node] = float('inf')
                type_matrix['time'][source_node][target_node] = float('inf')
                type_matrix['risk'][source_node][target_node] = float('inf')
                type_matrix['path'][source_node][target_node] = []
            except Exception as e:
                print(f"Error calculating path for {vehicle_type} from {source_node} to {target_node}: {e}")
                type_matrix['score'][source_node][target_node] = float('inf')
                type_matrix['time'][source_node][target_node] = float('inf')
                type_matrix['risk'][source_node][target_node] = float('inf')
                type_matrix['path'][source_node][target_node] = []

    vehicle_matrices[vehicle_type] = type_matrix
    print(f"  Matrix for {vehicle_type} complete.")

print("\n--- Path Matrix Calculation Complete ---\n")

In [ ]:
# Cell 9: Calculate Total Score Function
def calculate_total_score(current_vehicles, victims_list, hospitals_list, vehicle_matrices):
    """Calculates the total score for a given state (vehicle assignments and routes).
    Lower score is better.
    """
    total_score = 0
    visited_victims_count = 0
    # Reset capacities for calculation
    for h in hospitals_list:
        h.capacity = h.beginning
    for v in current_vehicles:
        v.current_amt = 0
        v.route = [] # Clear route before recalculating
        v.risk_scores = []
    for vic in victims_list:
        vic.visited = False

    # Assign victims to vehicles and build routes based on assigned_victims
    for vehicle in current_vehicles:
        current_loc = vehicle.location # Vehicle's starting location
        vehicle_type = vehicle.type
        
        # Sort assigned victims by rating to prioritize 'R' first (Red, then Unknown, then Green)
        # If ratings are equal, order doesn't strictly matter for initial assignment, but path order might.
        # For now, we'll process them in the order they were assigned during neighbor generation, which should be fine.
        # The scoring mechanism heavily penalizes unvisited 'R' victims.

        vehicle_path_score = 0
        path_segments = [] # To store (source, target) for route construction

        # First, ensure red victims are processed as highest priority
        red_victims_assigned = [v for v in vehicle.assigned_victims if v.rating == 'R']
        other_victims_assigned = [v for v in vehicle.assigned_victims if v.rating != 'R']
        
        # Prioritize red victims first, then others
        sorted_assigned_victims = red_victims_assigned + other_victims_assigned
        
        # Create a copy to iterate, allowing modification of original list if needed (though not done here)
        assigned_victims_copy = list(sorted_assigned_victims)

        for victim in assigned_victims_copy:
            if vehicle.current_amt < vehicle.capacity:
                try:
                    path_score = vehicle_matrices[vehicle_type]['score'][current_loc][victim.location]
                    travel_time = vehicle_matrices[vehicle_type]['time'][current_loc][victim.location]
                    risk_taken = vehicle_matrices[vehicle_type]['risk'][current_loc][victim.location]
                    path_nodes = vehicle_matrices[vehicle_type]['path'][current_loc][victim.location]
                    
                    if math.isinf(path_score): # Check if path is inaccessible
                        # If a path is inaccessible, this assignment is invalid, penalize heavily
                        total_score += float('inf')
                        continue # Skip to next vehicle or victim

                    vehicle_path_score += path_score
                    vehicle.current_amt += 1
                    victim.visited = True
                    visited_victims_count += 1
                    
                    # Add cost for victim rating based on travel time
                    if victim.rating == 'R':
                        vehicle_path_score += 0.5 * travel_time # Higher penalty for critical victims
                    elif victim.rating == 'U':
                        vehicle_path_score += 0.2 * travel_time # Medium penalty for unknown victims
                    
                    # Extend vehicle's route and risk scores
                    if not vehicle.route: # If this is the first segment, include current_loc
                        vehicle.route.extend(path_nodes)
                    else:
                        vehicle.route.extend(path_nodes[1:]) # Append from second node to avoid duplication
                    vehicle.risk_scores.append(risk_taken)
                    
                    current_loc = victim.location # Update current location for next segment
                except KeyError:
                    # This can happen if a node is isolated or an assignment is invalid
                    total_score += float('inf')
                    continue
            else: # Vehicle is full, must go to hospital
                # Find nearest hospital with capacity
                best_hospital = None
                min_hospital_score = float('inf')
                for h in hospitals_list:
                    if h.capacity > 0:
                        try:
                            hospital_score = vehicle_matrices[vehicle_type]['score'][current_loc][h.location]
                            if hospital_score < min_hospital_score:
                                min_hospital_score = hospital_score
                                best_hospital = h
                        except KeyError:
                            pass # No path to this hospital
                
                if best_hospital:
                    hospital_path_score = vehicle_matrices[vehicle_type]['score'][current_loc][best_hospital.location]
                    hospital_path_nodes = vehicle_matrices[vehicle_type]['path'][current_loc][best_hospital.location]
                    hospital_risk_taken = vehicle_matrices[vehicle_type]['risk'][current_loc][best_hospital.location]

                    vehicle_path_score += hospital_path_score
                    best_hospital.capacity -= vehicle.current_amt # Drop off all victims
                    vehicle.current_amt = 0 # Vehicle is now empty
                    
                    if not vehicle.route:
                        vehicle.route.extend(hospital_path_nodes)
                    else:
                        vehicle.route.extend(hospital_path_nodes[1:])
                    vehicle.risk_scores.append(hospital_risk_taken)
                    current_loc = best_hospital.location # Vehicle is now at hospital
    
    # After visiting victims and potentially going to a hospital, 
    # if the vehicle has victims, it must drop them off.
    # If the vehicle has visited victims and is not at a hospital, route to nearest hospital.
    if vehicle.current_amt > 0: # Vehicle still has victims
        best_hospital = None
        min_hospital_score = float('inf')
        for h in hospitals_list:
            if h.capacity > 0: # Ensure hospital has capacity
                try:
                    hospital_score = vehicle_matrices[vehicle_type]['score'][current_loc][h.location]
                    if hospital_score < min_hospital_score:
                        min_hospital_score = hospital_score
                        best_hospital = h
                except KeyError:
                    pass
        
        if best_hospital:
            hospital_path_score = vehicle_matrices[vehicle_type]['score'][current_loc][best_hospital.location]
            hospital_path_nodes = vehicle_matrices[vehicle_type]['path'][current_loc][best_hospital.location]
            hospital_risk_taken = vehicle_matrices[vehicle_type]['risk'][current_loc][best_hospital.location]

            vehicle_path_score += hospital_path_score
            best_hospital.capacity -= vehicle.current_amt
            vehicle.current_amt = 0
            
            if not vehicle.route:
                vehicle.route.extend(hospital_path_nodes)
            else:
                vehicle.route.extend(hospital_path_nodes[1:])
            vehicle.risk_scores.append(hospital_risk_taken)
            current_loc = best_hospital.location
        else:
            # No hospital with capacity found, penalize heavily
            vehicle_path_score += float('inf')

    # Add vehicle's total path score to the overall total
    total_score += vehicle_path_score

    # Penalize unvisited 'Red' victims most heavily
    for vic in victims_list:
        if vic.rating == 'R' and not vic.visited:
            total_score += 5000 # High penalty for unvisited critical victims

    return total_score

In [ ]:
# Cell 10: Generate Neighbor Function (for Simulated Annealing)
def get_neighbor(current_vehicles, victims_list, hospitals_list, vehicle_matrices):
    """Generates a new neighboring solution by slightly modifying the current one.
       This involves reassigning victims or swapping assignments between vehicles.
    """
    # Create deep copies to avoid modifying the original solution directly
    new_vehicles = []
    for v in current_vehicles:
        new_vehicle = Vehicle(v.type, v.max_risk, v.location, v.capacity, v.index, v.latlong)
        # Deep copy assigned_victims - ensure it's a list of new Victim objects
        new_vehicle.assigned_victims = []
        for old_victim in v.assigned_victims:
            new_victim_copy = Victim(old_victim.location, old_victim.rating, old_victim.latlong)
            new_vehicle.assigned_victims.append(new_victim_copy)
        new_vehicles.append(new_vehicle)

    new_victims = []
    victim_map = {vic.location: vic for vic in victims_list} # Map victim node to victim object
    for vic in victims_list:
        new_victims.append(Victim(vic.location, vic.rating, vic.latlong))

    new_hospitals = []
    for h in hospitals_list:
        new_hospitals.append(Hospital(h.location, h.capacity, h.latlong))

    # --- Perturbation Logic ---
    action = random.choice(['swap_victims_between_vehicles', 'move_victim_to_vehicle', 'swap_vehicle_assignments'])

    if action == 'swap_victims_between_vehicles' and len(new_vehicles) >= 2:
        # Attempt to swap victims between two randomly chosen vehicles
        v1, v2 = random.sample(new_vehicles, 2)
        if v1.assigned_victims and v2.assigned_victims:
            vic1_idx = random.randrange(len(v1.assigned_victims))
            vic2_idx = random.randrange(len(v2.assigned_victims))
            
            # Swap the victim objects directly (they are copies)
            temp_vic = v1.assigned_victims[vic1_idx]
            v1.assigned_victims[vic1_idx] = v2.assigned_victims[vic2_idx]
            v2.assigned_victims[vic2_idx] = temp_vic

    elif action == 'move_victim_to_vehicle' and new_victims and new_vehicles:
        # Attempt to move a random unassigned victim to a random vehicle, or reassign an assigned one
        target_victim = random.choice(new_victims)
        target_vehicle = random.choice(new_vehicles)
        
        # Remove victim from its current assignment if it's already assigned
        for v in new_vehicles:
            if target_victim in v.assigned_victims:
                v.assigned_victims.remove(target_victim)
                break
        
        # Assign victim to the new vehicle
        target_vehicle.assigned_victims.append(target_victim)

    elif action == 'swap_vehicle_assignments' and new_victims and len(new_vehicles) >= 2:
        # Pick two random victims and try to assign them to two random vehicles (possibly same)
        vic_to_assign1 = random.choice(new_victims)
        vic_to_assign2 = random.choice(new_victims)
        veh1, veh2 = random.sample(new_vehicles, 2)

        # Remove victims from their current assignments
        for v in new_vehicles:
            if vic_to_assign1 in v.assigned_victims:
                v.assigned_victims.remove(vic_to_assign1)
            if vic_to_assign2 in v.assigned_victims:
                v.assigned_victims.remove(vic_to_assign2)
        
        # Assign them to the new vehicles
        veh1.assigned_victims.append(vic_to_assign1)
        veh2.assigned_victims.append(vic_to_assign2)

    # IMPORTANT: Ensure 'R' (Red) victims are always assigned if possible.
    # This logic can be more sophisticated (e.g., ensure they are assigned first in a route).
    # For simplicity here, we'll just check if any 'R' victims are unassigned and assign them to a random available vehicle.
    assigned_victim_locations = set()
    for v in new_vehicles:
        assigned_victim_locations.update([vic.location for vic in v.assigned_victims])
    
    unassigned_red_victims = [vic for vic in new_victims if vic.rating == 'R' and vic.location not in assigned_victim_locations]
    
    for red_vic in unassigned_red_victims:
        if new_vehicles:
            # Assign to a random vehicle, preferably one that can handle it
            # A more intelligent assignment might pick the closest vehicle or least loaded
            chosen_vehicle = random.choice(new_vehicles)
            chosen_vehicle.assigned_victims.append(red_vic)
            assigned_victim_locations.add(red_vic.location)

    # --- Route Re-construction and Validation (Simplified) ---
    # The actual paths and scores will be recalculated in calculate_total_score
    # Here, we mainly update the assigned_victims list for each vehicle.
    # Sort assigned victims within each vehicle: prioritize 'R' victims, then others.
    for vehicle in new_vehicles:
        red_vics = sorted([v for v in vehicle.assigned_victims if v.rating == 'R'], 
                          key=lambda x: vehicle_matrices[vehicle.type]['score'].get(vehicle.location, {}).get(x.location, float('inf')))
        other_vics = sorted([v for v in vehicle.assigned_victims if v.rating != 'R'], 
                            key=lambda x: vehicle_matrices[vehicle.type]['score'].get(vehicle.location, {}).get(x.location, float('inf')))
        vehicle.assigned_victims = red_vics + other_vics # Red victims first

    return new_vehicles, new_victims, new_hospitals # Return modified copies

In [ ]:
# Cell 11: Simulated Annealing Algorithm
def simulated_annealing(vehicles_initial, victims_list, hospitals_list, vehicle_matrices, 
                        initial_temperature, cooling_rate, iterations):
    """Implements the simulated annealing metaheuristic for optimization."""
    
    # Make initial deep copies for the algorithm to work with
    current_vehicles = []
    for v in vehicles_initial:
        new_vehicle = Vehicle(v.type, v.max_risk, v.location, v.capacity, v.index, v.latlong)
        # No assigned victims yet for initial state, will be assigned by get_neighbor if it takes all victims
        current_vehicles.append(new_vehicle)

    current_victims = []
    for vic in victims_list:
        current_victims.append(Victim(vic.location, vic.rating, vic.latlong))

    current_hospitals = []
    for h in hospitals_list:
        current_hospitals.append(Hospital(h.location, h.capacity, h.latlong))

    # Initialize 'current_solution' to a random valid assignment first
    # This is a crucial step for SA - ensure the initial solution is reasonably valid.
    # Simple initial assignment: distribute victims round-robin or based on proximity
    # For now, we'll rely on `get_neighbor` to generate a sensible first state from a 'blank' initial state.

    # Start with a random initial assignment for the 'current_solution'
    temp_current_vehicles, temp_current_victims, temp_current_hospitals = 
        get_neighbor(current_vehicles, current_victims, current_hospitals, vehicle_matrices)
    
    # Distribute all victims initially to get a starting point
    unassigned_vics = list(temp_current_victims) # All victims are initially unassigned in this context
    random.shuffle(unassigned_vics)
    
    for i, vic in enumerate(unassigned_vics):
        if temp_current_vehicles:
            # Assign to vehicles in a round-robin fashion
            assigned_vehicle_idx = i % len(temp_current_vehicles)
            temp_current_vehicles[assigned_vehicle_idx].assigned_victims.append(vic)

    current_vehicles = temp_current_vehicles
    current_victims = temp_current_victims
    current_hospitals = temp_current_hospitals

    current_score = calculate_total_score(current_vehicles, current_victims, current_hospitals, vehicle_matrices)
    best_score = current_score
    best_vehicles = current_vehicles
    best_victims = current_victims # Store victims state associated with best vehicles
    
    temperature = initial_temperature

    print("Starting Simulated Annealing...")
    print(f"Initial Score: {current_score:.2f}")

    for i in range(iterations):
        # Generate a neighbor solution
        neighbor_vehicles, neighbor_victims, neighbor_hospitals = \
            get_neighbor(current_vehicles, victims_list, hospitals_list, vehicle_matrices)
        
        neighbor_score = calculate_total_score(neighbor_vehicles, neighbor_victims, neighbor_hospitals, vehicle_matrices)

        # Calculate energy difference
        delta_e = neighbor_score - current_score
        
        # Acceptance probability
        if delta_e < 0 or (temperature > 0 and random.uniform(0, 1) < math.exp(-delta_e / temperature)):
            current_score = neighbor_score
            current_vehicles = neighbor_vehicles
            current_victims = neighbor_victims
            current_hospitals = neighbor_hospitals
            
            if current_score < best_score:
                best_score = current_score
                best_vehicles = current_vehicles # Store the vehicles for the best solution
                best_victims = current_victims # Store victims state for consistency

        # Cool down the temperature
        temperature *= cooling_rate
        
        if i % (iterations // 10) == 0: # Print progress every 10% of iterations
            print(f"Iteration {i}/{iterations}, Temp: {temperature:.2f}, Current Score: {current_score:.2f}, Best Score: {best_score:.2f}")
        
        if temperature < 1e-5: # Stop if temperature gets too low
            print(f"Temperature too low, stopping at iteration {i}")
            break

    print("\nSimulated Annealing Finished.")
    print(f"Final Best Score: {best_score:.2f}")
    
    # Recalculate the route and visited status for the best solution before returning
    # This ensures the returned 'best_vehicles' has its 'route', 'risk_scores', 'current_amt' 
    # and associated 'best_victims' have their 'visited' status correctly set.
    final_score = calculate_total_score(best_vehicles, best_victims, hospitals_list, vehicle_matrices)

    return best_score, best_vehicles

In [ ]:
# Cell 12: Execute Simulated Annealing
# Ensure 'vehicles', 'victims', 'hospitals', and 'vehicle_matrices' are defined from previous cells.

initial_temp = 100
cooling_rate = 0.95
num_iterations = 10000

final_best_score, final_best_vehicles = simulated_annealing(
    vehicles,
    victims,
    hospitals,
    vehicle_matrices,
    initial_temp,
    cooling_rate,
    num_iterations
)

print("\n--- Optimized Solution Details ---")
print(f"Total Score: {final_best_score:.2f}")
for i, vehicle in enumerate(final_best_vehicles):
    print(f"\nVehicle {vehicle.index} ({vehicle.type}):")
    if vehicle.assigned_victims:
        print(f"  Assigned Victims: {[f'Victim at {v.location} ({v.rating})' for v in vehicle.assigned_victims]}")
        print(f"  Route (nodes): {vehicle.route}")
        print(f"  Risk Scores (per segment): {vehicle.risk_scores}")
    else:
        print("  No victims assigned.")

# You might want to add plotting code here to visualize the routes on the map
# Example (requires matplotlib, which is already imported by osmnx):
# fig, ax = ox.plot_graph_routes(G, [v.route for v in final_best_vehicles if v.route], 
#                                route_colors=['r', 'g', 'b', 'y'], route_linewidth=2, 
#                                node_size=0, bgcolor='k')
# for h in hospitals:
#     ax.scatter(G.nodes[h.location]['x'], G.nodes[h.location]['y'], color='blue', marker='s', s=100, label='Hospital')
# for v in victims:
#     color = 'red' if v.rating == 'R' else ('orange' if v.rating == 'U' else 'green')
#     ax.scatter(G.nodes[v.location]['x'], G.nodes[v.location]['y'], color=color, marker='o', s=50, label=f'Victim ({v.rating})')
# plt.show()
